In [2]:
# setup commands

%load_ext autoreload
%autoreload 2

In [5]:
import yoga_dataset
import os
import numpy as np


data_path = 'data'
count = 0
for file in os.listdir(data_path):
    file_path = os.path.join(data_path, file)
    arr = np.load(file_path)
    print(arr.shape)
    count += 1

print(count)

(66, 23, 3)
(76, 23, 3)
(356, 23, 3)
(236, 23, 3)
(205, 23, 3)
(254, 23, 3)
(277, 23, 3)
(375, 23, 3)
(286, 23, 3)
(269, 23, 3)
(709, 23, 3)
(135, 23, 3)
(290, 23, 3)
(264, 23, 3)
(270, 23, 3)
(45, 23, 3)
(64, 23, 3)
(275, 23, 3)
(268, 23, 3)
(173, 23, 3)
(74, 23, 3)
(504, 23, 3)
(501, 23, 3)
(192, 23, 3)
(174, 23, 3)
(266, 23, 3)
(207, 23, 3)
(117, 23, 3)
(209, 23, 3)
(138, 23, 3)
(301, 23, 3)
(234, 23, 3)
(184, 23, 3)
(137, 23, 3)
(151, 23, 3)
(123, 23, 3)
(166, 23, 3)
(204, 23, 3)
(208, 23, 3)
(370, 23, 3)
(239, 23, 3)
(37, 23, 3)
(120, 23, 3)
(49, 23, 3)
(69, 23, 3)
(281, 23, 3)
(129, 23, 3)
(253, 23, 3)
(150, 23, 3)
(206, 23, 3)
(123, 23, 3)
(214, 23, 3)
(310, 23, 3)
(92, 23, 3)
(59, 23, 3)
(172, 23, 3)
(282, 23, 3)
(44, 23, 3)
(288, 23, 3)
(70, 23, 3)
(168, 23, 3)
(358, 23, 3)
(123, 23, 3)
(288, 23, 3)
(14, 23, 3)
(302, 23, 3)
(279, 23, 3)
(133, 23, 3)
(226, 23, 3)
(279, 23, 3)
(64, 23, 3)
(217, 23, 3)
(138, 23, 3)
(72, 23, 3)
(160, 23, 3)
(240, 23, 3)
(158, 23, 3)
(171, 23, 3)
(

In [19]:
import torch
from cnn import CNN


input = torch.randn(10, 3, 23)
model = CNN([3, 16, 32, 64, 64])
out = model(input)
print(out.shape)

torch.Size([10, 64, 23])
torch.Size([10, 82])
